 # Install/Import Packages and Download/Unzip Dataset

 ---

 The following code installs d3rlpy, imports all necessary dependencies, and downloads/unzips the dataset from Zenodo.

In [ ]:
!pip install d3rlpy==2.6.1 gymnasium==0.29.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827621 sha256=6cb2d988b41f40384a31e2b94e56c26f201f0d1939eb3cf14bcce5d3226aee2b
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not curr

In [ ]:
from google.colab import files
import numpy as np
import d3rlpy
import os

In [ ]:
!wget -O full_dataset.zip "https://zenodo.org/record/13830810/files/full_dataset.zip?download=1"

--2024-09-25 20:52:48--  https://zenodo.org/record/13830810/files/full_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/13830810/files/full_dataset.zip [following]
--2024-09-25 20:52:49--  https://zenodo.org/records/13830810/files/full_dataset.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2161582746 (2.0G) [application/octet-stream]
Saving to: ‘full_dataset.zip’

full_dataset.zip    100%[===================>]   2.01G  24.1MB/s    in 87s     

2024-09-25 20:54:16 (23.6 MB/s) - ‘full_dataset.zip’ saved [2161582746/2161582746]



In [ ]:
!unzip full_dataset.zip -d /content/
data_dir = '/content/content/data/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/data/data/ep_38406.npz  
  inflating: /content/content/data/data/ep_45886.npz  
  inflating: /content/content/data/data/ep_80720.npz  
  inflating: /content/content/data/data/ep_41811.npz  
  inflating: /content/content/data/data/ep_74361.npz  
  inflating: /content/content/data/data/ep_36142.npz  
  inflating: /content/content/data/data/ep_92647.npz  
  inflating: /content/content/data/data/ep_35769.npz  
  inflating: /content/content/data/data/ep_70452.npz  
  inflating: /content/content/data/data/ep_89001.npz  
  inflating: /content/content/data/data/ep_92324.npz  
  inflating: /content/content/data/data/ep_21612.npz  
  inflating: /content/content/data/data/ep_51788.npz  
  inflating: /content/content/data/data/ep_51083.npz  
  inflating: /content/content/data/data/ep_54442.npz  
  inflating: /content/content/data/data/ep_97628.npz  
  inflating: /content/content/data/data/ep_26720.npz  
  inflating: /

# Prepare Dataset for d3rlpy

---

The following code loads all of the data into arrays and creates a d3rlpy dataset.

In [ ]:
total_steps = 5000000

all_observations = np.empty((total_steps, 3, 64, 64), dtype=np.uint8)
all_actions = np.empty((total_steps, 3), dtype=np.float32)
all_rewards = np.empty((total_steps,), dtype=np.float32)
all_terminals = np.empty((total_steps,), dtype=bool)

current_index = 0

for i in range(1, 25001):
    episode_file = os.path.join(data_dir, f'ep_{i}.npz')
    episode_data = np.load(episode_file)

    num_steps = episode_data['terminals'].shape[0]

    all_observations[current_index:current_index + num_steps] = np.transpose(episode_data['observations'], (0, 3, 1, 2))
    all_actions[current_index:current_index + num_steps] = episode_data['actions']
    all_rewards[current_index:current_index + num_steps] = episode_data['rewards']
    all_terminals[current_index:current_index + num_steps] = episode_data['terminals']

    current_index += num_steps

all_observations = all_observations[:current_index]
all_actions = all_actions[:current_index]
all_rewards = all_rewards[:current_index]
all_terminals = all_terminals[:current_index]

In [ ]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=all_observations,
    actions=all_actions,
    rewards=all_rewards,
    terminals=all_terminals,
)

2024-09-25 21:01.10 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-09-25 21:01.10 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-09-25 21:01.10 [info     ] Action size has been automatically determined. action_size=3


# Model Training

---

The following code trains a behavior cloning algorithm for 50 epochs at 10000 steps per epoch.

In [ ]:
BC = d3rlpy.algos.BCConfig(observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),).create(device='cuda')
BC.fit(dataset=dataset, n_steps=500000, n_steps_per_epoch=10000)

2024-09-25 21:01.11 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2024-09-25 21:01.11 [info     ] Directory is created at d3rlpy_logs/BC_20240925210111
2024-09-25 21:01.11 [debug    ] Building models...            
2024-09-25 21:01.13 [debug    ] Models have been built.       
2024-09-25 21:01.13 [info     ] Parameters                     params={'observation_shape': [3, 64, 64], 'action_size': 3, 'config': {'type': 'bc', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'pixel', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 0.001, 'policy_type': 'deterministic', 'optim_factory': {'type': '

Epoch 1/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:02.33 [info     ] BC_20240925210111: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.0028660141229629515, 'time_algorithm_update': 0.005022420930862427, 'loss': 0.01563481689346954, 'time_step': 0.007979437351226807} step=10000
2024-09-25 21:02.33 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925210111/model_10000.d3


Epoch 2/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:03.52 [info     ] BC_20240925210111: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.002807782959938049, 'time_algorithm_update': 0.004874714803695679, 'loss': 0.008621751333330758, 'time_step': 0.007771308946609497} step=20000
2024-09-25 21:03.52 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925210111/model_20000.d3


...

Epoch 49/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 22:05.28 [info     ] BC_20240925210111: epoch=49 step=490000 epoch=49 metrics={'time_sample_batch': 0.002823265266418457, 'time_algorithm_update': 0.004897894525527954, 'loss': 0.003419490486430004, 'time_step': 0.007809411835670471} step=490000
2024-09-25 22:05.28 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925210111/model_490000.d3


Epoch 50/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 22:06.46 [info     ] BC_20240925210111: epoch=50 step=500000 epoch=50 metrics={'time_sample_batch': 0.002828282880783081, 'time_algorithm_update': 0.004894616913795471, 'loss': 0.003426086270972155, 'time_step': 0.0078120455503463745} step=500000
2024-09-25 22:06.47 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925210111/model_500000.d3


[(1,
  {'time_sample_batch': 0.0028660141229629515,
   'time_algorithm_update': 0.005022420930862427,
   'loss': 0.01563481689346954,
   'time_step': 0.007979437351226807}),
 (2,
  {'time_sample_batch': 0.002807782959938049,
   'time_algorithm_update': 0.004874714803695679,
   'loss': 0.008621751333330758,
   'time_step': 0.007771308946609497}),
 (3,
  {'time_sample_batch': 0.0028023838996887205,
   'time_algorithm_update': 0.004854305624961853,
   'loss': 0.00738925339858979,
   'time_step': 0.007745253396034241}),
 (4,
  {'time_sample_batch': 0.0028173617601394653,
   'time_algorithm_update': 0.004886588954925537,
   'loss': 0.0066413570678792895,
   'time_step': 0.007792285108566284}),
 (5,
  {'time_sample_batch': 0.0028092530012130737,
   'time_algorithm_update': 0.004891440606117249,
   'loss': 0.006154018088849261,
   'time_step': 0.007789110708236694}),
 (6,
  {'time_sample_batch': 0.0028082342863082884,
   'time_algorithm_update': 0.004899337792396545,
   'loss': 0.005777203653

In [ ]:
!zip -r model_quarter_dataset.zip d3rlpy_logs

  adding: d3rlpy_logs/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925210111/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925210111/model_90000.d3 (deflated 33%)
  adding: d3rlpy_logs/BC_20240925210111/time_algorithm_update.csv (deflated 61%)
  adding: d3rlpy_logs/BC_20240925210111/time_step.csv (deflated 60%)
  adding: d3rlpy_logs/BC_20240925210111/model_30000.d3 (deflated 21%)
  adding: d3rlpy_logs/BC_20240925210111/model_320000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_440000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_340000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_250000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_160000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_290000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_130000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_500000.d3 (deflated 36%)
  adding: d3rlpy_logs/BC_20240925210111/model_140000.d